In [7]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
elastic_rods.set_max_num_tbb_threads(4)
l = elastic_rods.RodLinkage('../../examples/florin/20181008_103824_meshID_95a3e4ba-e509-432a-9442-738b53a06248.obj', 5)
driver=l.centralJoint()
mat = elastic_rods.RodMaterial('+', 2000, 0.3, [2.0, 2.0, 0.2, 0.2])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
#with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.setCameraParams(((-2.200522340164312, 0.5782056167253651, -0.7549139983294457),
 (-0.02031290489160542, -0.03507141600564204, -0.999178353285445),
 (-0.17091934654983643, 0.17173250980468135, 0.09235104577312339)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [3]:
elastic_rods.compute_equilibrium(l, fixedVars=list(range(jdo, jdo + 6)))
view.update()

0	0.886261	0.234163	0.234163	0.0625	0
1	0.799898	2.42588	2.42588	1	0
2	0.706927	6.36529	6.36529	1	0
3	0.540713	1.24332	1.24332	0.5	0
4	0.487166	3.30869	3.30869	1	0
5	0.408138	1.77182	1.77182	1	0
6	0.401263	5.51536	5.51536	1	0
7	0.290235	1.81085	1.81085	0.5	0
8	0.27851	3.92343	3.92343	1	0
9	0.208863	1.11021	1.11021	0.5	0
10	0.192415	2.57785	2.57785	1	0
11	0.152093	1.25928	1.25928	0.5	0
12	0.134504	1.47776	1.47776	1	0
13	0.110587	1.12052	1.12052	1	0
14	0.0945185	1.60288	1.60288	1	0
15	0.0767771	1.08254	1.08254	1	0
16	0.0683544	1.51294	1.51294	1	0
17	0.0572806	1.11465	1.11465	1	0
18	0.0494807	0.61751	0.61751	1	0
19	0.0478018	0.946013	0.946013	1	0
20	0.043321	0.126103	0.126103	1	0
21	0.0428805	0.260781	0.260781	1	0
22	0.0425623	0.00609503	0.00609503	1	0
23	0.0425571	0.00246248	0.00246248	1	0
24	0.0425571	5.47837e-07	5.47837e-07	1	0
25	0.0425571	3.20431e-09	3.20431e-09	1	0


In [2]:
restLinkage = elastic_rods.RodLinkage(l)
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.useIdentityMetric = True
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [6]:
# report = open_linkage(l, driver, np.pi/2, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=False);
report = open_linkage(l, driver, np.pi/4, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=True);

target angle:  0.27717146445513974
0	11.4692	46.9248	46.9248	1	0
CHOLMOD detected non-positive definite matrix!; increasing tau to 1e-08
CHOLMOD detected non-positive definite matrix!; increasing tau to 4e-08
CHOLMOD detected non-positive definite matrix!; increasing tau to 1.6e-07
CHOLMOD detected non-positive definite matrix!; increasing tau to 6.4e-07
CHOLMOD detected non-positive definite matrix!; increasing tau to 2.56e-06
CHOLMOD detected non-positive definite matrix!; increasing tau to 1.024e-05
CHOLMOD detected non-positive definite matrix!; increasing tau to 4.096e-05
1	1.1437	0.0236296	0.0236296	1	1
CHOLMOD detected non-positive definite matrix!; increasing tau to 2.048e-05
2	1.14268	0.00858645	0.00858645	1	1
CHOLMOD detected non-positive definite matrix!; increasing tau to 1.024e-05
3	1.14267	0.000294517	0.000294517	1	1
CHOLMOD detected non-positive definite matrix!; increasing tau to 5.12e-06
CHOLMOD detected non-positive definite matrix!; increasing tau to 2.048e-05
4	1.14

In [ ]:
with suppress_stdout():
    l = elastic_rods.RodLinkage(restLinkage)
    view.update(l=l)
    report_single = open_linkage(l, driver, np.pi/2, 400, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=50, verbose=False, useTargetAngleConstraint=False)
    deployed_single = elastic_rods.RodLinkage(l)
    l = elastic_rods.RodLinkage(restLinkage)
    view.update(l=l)
    report_global = open_linkage(l, driver, np.pi/2, 400, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=50, verbose=False, useTargetAngleConstraint=True)
    deployed_global = elastic_rods.RodLinkage(l)

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.plot(report_single[1])
plt.title('Single joint actuation')
plt.ylabel('Actuation torque')
plt.xlabel('Step')
plt.show()
plt.plot(report_global[1])
plt.title('Global actuation')
plt.ylabel('Actuation torque')
plt.xlabel('Step')
plt.show()

In [ ]:
view_single = LinkageViewer(deployed_single)
view_single.show()

In [ ]:
view_global.setCameraParams(view_single.getCameraParams())

In [ ]:
view_global = LinkageViewer(deployed_global)
view_global.show()